In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [ ]:
dataset, metadata = tfds.load("fashion_mnist", as_supervised = True, with_info = True)
train_dataset, test_dataset = dataset["train"], dataset["test"]


In [ ]:
train_dataset, test_dataset = train_dataset.cache(), test_dataset.cache()

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']

In [ ]:
def normalize(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255
  return image, label

train_dataset = train_dataset.map(normalize)
test_dataset = test_dataset.map(normalize)

In [ ]:
n_row, n_col = 5, 5
fig = plt.figure(figsize=(10, 10))



for idx, (image, label) in enumerate(test_dataset.take(25)):
  image = image.numpy().reshape((28, 28))
  plt.subplot(5,5,idx + 1)
  plt.imshow(image, cmap="gray")
  plt.xticks([])
  plt.yticks([])
  plt.xlabel(class_names[label])
  plt.grid(False)

In [ ]:
num_samples_training = metadata.splits["train"].num_examples
num_samples_test = metadata.splits["test"].num_examples
print(f"Training samples: {num_samples_training}\nTest samples: {num_samples_test}")

In [ ]:
BATCH_SIZE = 32
epochs = 10

train_dataset = train_dataset.cache().repeat().shuffle(num_samples_training).batch(BATCH_SIZE)
test_dataset = test_dataset.cache().batch(BATCH_SIZE)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

In [ ]:
model = Sequential(layers=[
    Conv2D(32, (3, 3), padding="same", activation="relu", input_shape=(28, 28, 1)),
    MaxPooling2D((2,2), strides=2),
    Conv2D(64, (3, 3), padding='same', activation="relu"),
    MaxPooling2D((2,2), strides=2),
    Flatten(),
    Dense(128, activation="relu"),
    Dense(64, activation="relu"),
    Dense(10, activation="softmax"),
])

In [ ]:
model.compile(optimizer = "adam", metrics=["accuracy"], loss="sparse_categorical_crossentropy")

In [ ]:
history = model.fit(train_dataset, epochs=epochs, steps_per_epoch=int(num_samples_training/BATCH_SIZE))

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset, steps=int(num_samples_test))
print(f"Accuracy: {test_accuracy}")

In [ ]:
for test_image, test_label in test_dataset.take(1):
  test_image = test_image.numpy()
  test_label = test_label.numpy()
  predict_image = model.predict(test_image)

In [ ]:
np.argmax(predict_image[1]), test_label[1]
predict_image[1, 4]

In [ ]:
def plot_image(i, predict_array, image, true_label):
  predict_array, img, true_label = predict_array[i], image[i], true_label[i]

  predict_label = np.argmax(predict_array)
  plt.imshow(img[:], cmap="gray")
  if predict_label == true_label:
    color = "blue"
  else:
    color = "red"

  plt.xlabel("{} {:2.0f}% : (True {})".format(class_names[predict_label],
                                              predict_array[predict_label] * 100,
                                              class_names[true_label]),
             color=color)

  plt.xticks([])
  plt.yticks([])
  plt.grid(False)

def plot_predict_percent(i, predict_array, true_label):
  predict_array, true_label = predict_array[i], true_label[i]

  predict_label = np.argmax(predict_array)

  plot = plt.bar(range(10), predict_array, color="#e5e7eb")
  plt.ylim([0, 1])
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plot[predict_label].set_color("red")
  plot[true_label].set_color("blue")

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1, 2, 1)
plot_image(i, predict_image, test_image, test_label)
plt.subplot(1, 2, 2)
plot_predict_percent(i, predict_image, test_label)

In [ ]:
n_row, n_col = 3, 3
plt.figure(figsize=(10, 10))

for i in range(10):
  plt.subplot(n_row, n_col, i)